# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,288.22,81,100,0.50,AU,1736366635
1,1,forbes,-33.3833,148.0167,290.11,68,50,1.36,AU,1736366636
2,2,beipiao,41.7919,120.7792,259.16,43,29,5.94,CN,1736366637
3,3,minas de marcona,-15.2119,-75.1103,294.90,58,53,7.41,PE,1736366640
4,4,anloga,5.7947,0.8973,300.83,83,8,4.45,GH,1736366641


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
#%%capture --no-display

# Configure the map plot
plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    size = "Humidity",
    xlabel = "Longitude",
    ylabel = "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 650,
    frame_height = 500
)

# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
sample_df = city_data_df.loc[(city_data_df["Cloudiness"] >= 80) & (city_data_df["Humidity"] >= 20) & (city_data_df["Humidity"] <= 40)]

# Drop any rows with null values
sample_df = sample_df.dropna()

# Display sample data
sample_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,sao vicente,-23.9631,-46.3919,297.58,26,100,4.12,BR,1736366521
133,133,at taj,24.2049,23.2857,287.39,38,100,2.21,LY,1736366833
166,166,zhicheng,30.2956,111.5047,277.44,34,100,0.96,CN,1736366873
172,172,naracoorte,-36.9579,140.7383,296.16,37,100,3.06,AU,1736366879
247,247,etzatlan,20.7667,-104.0833,295.70,34,100,1.92,MX,1736366968
266,266,bardai,11.6500,7.2333,293.31,22,81,4.33,NG,1736366938
280,280,necochea,-38.5473,-58.7368,303.40,33,100,6.64,AR,1736366775
304,304,ain sefra,32.7500,-0.5833,284.08,39,100,1.59,DZ,1736367035
348,348,qamdo,31.1667,97.2333,262.66,39,87,0.88,CN,1736367090
416,416,san julian,21.0167,-102.1667,292.74,35,100,5.38,MX,1736367158


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = sample_df[["City", "Country", "Lng", "Lat", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\eessd\AppData\Local\Temp\ipykernel_51996\720272543.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lng,Lat,Humidity,Hotel Name
60,sao vicente,BR,-46.3919,-23.9631,26,
133,at taj,LY,23.2857,24.2049,38,
166,zhicheng,CN,111.5047,30.2956,34,
172,naracoorte,AU,140.7383,-36.9579,37,
247,etzatlan,MX,-104.0833,20.7667,34,
266,bardai,NG,7.2333,11.6500,22,
280,necochea,AR,-58.7368,-38.5473,33,
304,ain sefra,DZ,-0.5833,32.7500,39,
348,qamdo,CN,97.2333,31.1667,39,
416,san julian,MX,-102.1667,21.0167,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "limit" : 20,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    #print(params)

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    #pprint.pprint(name_address)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sao vicente - nearest hotel: Pousada Vitória
at taj - nearest hotel: No hotel found
zhicheng - nearest hotel: No hotel found
naracoorte - nearest hotel: Kincraig Hotel /Smith & Jones Cafe
etzatlan - nearest hotel: Hotel Cuatro Caminos
bardai - nearest hotel: No hotel found
necochea - nearest hotel: Cabañas Río Quequén
ain sefra - nearest hotel: فندق مكثر
qamdo - nearest hotel: 银杏商务宾馆
san julian - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
wuhai - nearest hotel: Binhe International Hotel


,City,Country,Lng,Lat,Humidity,Hotel Name
60,sao vicente,BR,-46.3919,-23.9631,26,Pousada Vitória
133,at taj,LY,23.2857,24.2049,38,No hotel found
166,zhicheng,CN,111.5047,30.2956,34,No hotel found
172,naracoorte,AU,140.7383,-36.9579,37,Kincraig Hotel /Smith & Jones Cafe
247,etzatlan,MX,-104.0833,20.7667,34,Hotel Cuatro Caminos
266,bardai,NG,7.2333,11.6500,22,No hotel found
280,necochea,AR,-58.7368,-38.5473,33,Cabañas Río Quequén
304,ain sefra,DZ,-0.5833,32.7500,39,فندق مكثر
348,qamdo,CN,97.2333,31.1667,39,银杏商务宾馆
416,san julian,MX,-102.1667,21.0167,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
#%%capture --no-display

# Configure the map plot
plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    size = "Humidity",
    xlabel = "Longitude",
    ylabel = "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 650,
    frame_height = 500,
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)